# **Tarea 5 - Transformers y BERT 📚**

## **Preguntas teóricas 📕 (3 puntos).** ##
Para estas preguntas no es necesario implementar código, pero pueden utilizar pseudo código.

### **Parte 1: Arquitecturas de Redes Neuronales**

**Pregunta 1**:

Explique el principal problema de las redes Elman recurrentes. Explique cada compuerta de las redes LSTM y la GRU.  **(0.75 puntos)**

>Las RNN Elman sufren el mismo problema que suelen tener algunas redes profundas: desvanecimiento del gradiente. Debido a la naturaleza recursiva de este tipo de red, existen muchas multiplicaciones que involucran valores $|w| < 1$, provocando que eventualmente los valores de desvanezcan. Por el contrario, si existe algún $|w| > 1$, recurrencias muy largas pueden producir el efecto de _exploding gradients_.
>
>Para compensar estos problemas, existen redes recurrentes que implementan _compuertas_, tales como las redes LSTM y GRU. Las redes LSTM utilizan 3 compuertas, 2 de ellas encargadas de actualizar la memoria de la red y 1 de filtrar cuánta memoria se transmite hacia el output.
>- Input: Determina cuánto considerar del input actual para la memoria de la red.
>- Forget: Determina cuánto considerar de la memoria anterior para la actual memoria.
>- Output: Una vez calculada la memoria de la red, esta compuerta determina cuánto de esa memoria se utiliza en el output de la red.
>
>Las redes GRU surgieron posterior a las redes LSTM, con el objetivo de disminuir la complejidad de este último tipo de red. Por lo mismo, utiliza sólo 2 compuertas:
>- Compuerta $r$: Utilizada para encontrar un candidato a la actualización del estado de la red.
>- Compuerta $z$: Usada para obtener el output de la red, al interpolar el estado anterior con el estado propuesto.

**Pregunta 2**:

Explique cuales son las diferencias entre las tres arquitecturas de sequence to sequence vistas en clases (Encoder-Decoder con RNN, Encoder-Decoder con RNN y Attention, y el Transformer) **(0.75 ptos)**

>Una arquitectura Encoder-Decoder RNN sin atención primero utiliza una red RNN como encoder, en la que una secuencia de tokens se usan como inputs y luego sólo se considera el output final $c_n$, el cual condensa toda la información de la cadena de tokens. A continuación una segunda RNN se usa como decoder, y concatena el input de la red con el output final del encoder, para luego obtener una salida y concatenarla con el output del encoder hasta el fin de la secuencia.
>
>El problema con este tipo de arquitecturas es que no se puede condensar toda la información en sólo un vector, lo que resulta problemático cuando se quiere utilizar una cadena larga de texto. Para solucionar este problema se puede usar atención, el que consiste en la utilización de una red biRNN como encoder, generando una secuencia de vectores $c_{i:n}$. En cuanto al decoder, para cada input de la secuencia ya no se usa solamente el output final del encoder, sino que una combinación convexa de los vectores $c_{i:n}$. La elección de los pesos $\alpha$ de esta combinación dependen del tipo de atención que se elija, pero dependen del estado del decoder y de toda la secuencia $c_{i:n}$. Gracias a este mecanismo de atención, el decoder puede enfocarse en distintas partes del encoder dependiendo del momento en que se encuentre.
>
>La gran diferencia de Transformers respecto a las arquitecturas anteriores, es que transformers ya no usa redes recurrentes. En su lugar, usa de forma inteligente una capa de atención para realizar el cómputo de toda la secuencia en paralelo, evitando la necesidad de calcular el input anterior de la secuencia para obtener el output del actual. Para ello utiliza 3 matrices: _key_, _value_ y _query_ que juntas logran encapsular toda la información necesaria. Por medio de las matrices key y query se calcula un score para cada token de la secuencia, el cual se multiplica por la matriz value para obtener una representación del token ponderado por la importancia del mismo. Al sumar todas estas representaciones se obtiene el output del encoder. El decoder tiene un funcionamiento similar al encoder, siendo una diferencia la utilización de 2 mecanismos de atención: uno de ellos permite prestar atención a toda la secuencia, mientras que otro sólo a las partes importantes, de forma similar a cómo se ha explicado para el encoder.

**Pregunta 3**:

¿Cúal es el principal diferencia entre los Embeddings contextualizados (por ejemplo BERT) vs. los Embeddings estáticos (por ejemplo Word2Vec)? **(0.75 ptos)**

>Los embeddings contextualizados generan embeddings de un token de acuerdo a la secuencia de tokens en la que aparecen. Por otra parte, los embeddings estáticos generan siempre el mismo embedding para un token, independientemente del contexto en el que aparezcan. Por extensión, al poseer los tokens un único embedding, cada uno de ellos posee solo un significado, cuando en realidad cada palabra puede poseer múltiples significados o comportamiento de acuerdo a la sintaxis.

**Pregunta 4**:

Explique en que tareas y las arquitecturas con las que se entrenan ELMO y BERT **(0.75 ptos)**

>ELMo se entrena en una tarea de Natural Language Model (NLM), la cual predice una palabra en base a la palabra anterior. El motivo detrás de la elección de esta tarea radica en que, al usar como input la palabra actual, la red en su estado intermedio creará una representación de esta palabra junto al contexto en el que se encuentra. Si en lugar de enfocarse en la tarea misma y se extraen estos vectores intermedios, es posible crear embeddings contextuales.
>
> La arquitectura de ELMo es un stack de 2 redes biLSTM. Para los embeddings iniciales se una una red CNN a nivel de caracteres, 2048 filtros de n-gramas de caracteres, 2 highway layers y una dimensionalidad de 512. Los estados constan de 4096 dimensiones y posee conexiones residuales.
>
>BERT se entrena para una tarea llamada Masked Language Modeling (MLM), similar a un modelo de lenguaje. La diferencia se encuentra en que en MLM, en lugar de predecir la siguiente palabra se enmascara una porcentaje del input, y se le pide a la red que reconstruya la secuencia. También se incorpota una segunda tarea que trata sobre _next sentence prediction_, en la que se le entrega al modelo 2 oraciones, y predice si la segunda oración sigue de la primera.
>
>A diferencia de ELMo, BERT se entrena usando un Transformer, obteniendo los embedding de los outputs del encoder. Existen originalmente 2 modelos: BERT-Base de 12 capas, 768 unidaes ocultas y 12 cabezas de atención. El otro modelo es BERT-Large, con 24 capas, 1024 unidades ocultas y 16 cabezas de atención.

## **Preguntas prácticas 💻 (3 puntos).** ##

### BERT

Lo primero es instalar las librerías necesarias.

In [2]:
from transformers import BertTokenizer, BertForNextSentencePrediction, BertForMaskedLM, BertForQuestionAnswering
from torch.nn import functional as F
import torch

Para las preguntas que siguen, utilizaremos distintas variantes de BERT disponibles en la librería transformers. [Aquí](https://huggingface.co/transformers/model_doc/bert.html) pueden encontrar toda la documentación necesaria. El modelo pre-entrenado a utilizar es "bert-base-uncased" (salvo para question answering).

BERT es un modelo de lenguaje que fue entrenado exhaustivamente sobre dos tareas: 1) Next sentence prediction. 2) Masked language modeling.

#### **BertForNextSentencePrediction**

**Pregunta 1 (1 pto en total):**  Utilizando el modelo BertForNextSentencePrediction de la librería transformers, muestre cual de las 2 oraciones es **más probable** que sea una continuación de la primera. Para esto defina la función $oracion\_mas\_probable$, que recibe el inicio de una frase, las alternativas para continuar esta frase y retorna un string indicando cual de las dos oraciones es más probable **(0.25 puntos cada una)**.

Por ejemplo:

Initial: "The sky is blue."\
A: "This is due to the shorter wavelength of blue light."\
B: "Chile is one of the world's greatest economies."

Debería retornar "La oración que continúa más probable es A", justificándolo con la evaluación de BERT.



In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

In [31]:
def oracion_mas_probable(first, sentA, sentB):
    #Tu implementacion
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
    
    # Tokenize the sentences
    tokenized_A = tokenizer.encode_plus(first, sentA, return_tensors='pt')
    tokenized_B = tokenizer.encode_plus(first, sentB, return_tensors='pt')
    
    # Calculate the probabilities
    logits_A = model(**tokenized_A).logits
    probs_A = F.softmax(logits_A, dim=1)
    
    logits_B = model(**tokenized_B).logits
    probs_B = F.softmax(logits_B, dim=1)
    
    # Return the sentence with the higher probability
    if probs_A[0,0] > probs_B[0,0]:
        highest_prob = "A"
    else:
        highest_prob = "B"
    
    return f"La oración que continúa más probable es {highest_prob}. Probabilidades: (A: {probs_A[0,0]}, B: {probs_B[0,0]})"

1.1)
Initial: "My cat is fluffy."\
A: "My dog has a curling tail."\
B: "A song can make or ruin a person’s day if they let it get to them."

In [32]:
oracion_mas_probable(
    "My cat is fluffy.",
    "My dog has a curling tail.",
    "A song can make or ruin a person’s day if they let it get to them."
)

'La oración que continúa más probable es A. Probabilidades: (A: 0.9999964237213135, B: 0.0025219712406396866)'

1.2)
Initial: "The Big Apple is famous worldwide."\
A: "You can add cinnamon for the perfect combination."\
B: "It is America's largest city."

In [33]:
oracion_mas_probable(
    "The Big Apple is famous worldwide.",
    "You can add cinnamon for the perfect combination.",
    "It is America's largest city."
)

'La oración que continúa más probable es B. Probabilidades: (A: 0.9999865293502808, B: 0.9999897480010986)'

1.3)
Initial: "Roses are red."\
A: "Violets are blue."\
B: "Fertilize them regularly for impressive flowers."

In [34]:
oracion_mas_probable(
    "Roses are red.",
    "Violets are blue.",
    "Fertilize them regularly for impressive flowers."
)

'La oración que continúa más probable es A. Probabilidades: (A: 0.9999957084655762, B: 0.9999926090240479)'

1.4)
Initial: "I play videogames the whole day."\
A: "They make me happy."\
B: "They make me rage."\

In [35]:
oracion_mas_probable(
    "I play videogames the whole day.",
    "They make me happy.",
    "They make me rage."
)

'La oración que continúa más probable es A. Probabilidades: (A: 0.9999908208847046, B: 0.9999867677688599)'

#### **BertForMaskedLM**

**Pregunta 2 (1 pto en total):**  Ahora utilizaremos BertForMaskedLM para **predecir una palabra oculta** en una oración. **(0.25 puntos cada una)**\
Por ejemplo:\
BERT input: "I want to _ a new car."\
BERT prediction: "buy"

In [36]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [159]:
def palabra_mas_probable(sentence):
    #Tu implementacion
    # Index of the [MASK] token
    mask_token_idx = 103

    # Tokenize the sentence
    indexed_tokens = tokenizer.encode(sentence, add_special_tokens=False)
    tokens_tensor = torch.Tensor([indexed_tokens]).long()

    # Find the index of the [MASK] token in the sentence
    masked_idx = (tokens_tensor.flatten() == mask_token_idx).nonzero().item()

    # Makes a prediction
    predictions = model(tokens_tensor)

    # Find the token with the highest probability
    pred_idx = torch.argmax(predictions[0][0], dim=1)[masked_idx].item()
    pred_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]

    print(f"""BERT input: "{sentence}"\nBERT prediction: "{pred_token}\"""")
    return pred_token

2.1)
BERT input: "[CLS] I love [MASK] . [SEP]"

In [160]:
palabra_mas_probable("[CLS] I love [MASK]. [SEP]")

BERT input: "[CLS] I love [MASK]. [SEP]"
BERT prediction: "you"


'you'

2.2)
BERT input: "[CLS] I hear that Karen is very [MASK] . [SEP]"

In [161]:
palabra_mas_probable("[CLS] I hear that Karen is very [MASK] . [SEP]")

BERT input: "[CLS] I hear that Karen is very [MASK] . [SEP]"
BERT prediction: "upset"


'upset'

2.3)
BERT input: "[CLS] She had the gift of being able to [MASK] . [SEP]"

In [162]:
palabra_mas_probable("[CLS] She had the gift of being able to [MASK] . [SEP]")

BERT input: "[CLS] She had the gift of being able to [MASK] . [SEP]"
BERT prediction: "fly"


'fly'

2.4)
BERT input: "[CLS] It's not often you find a [MASK] on the street. [SEP]"

In [163]:
palabra_mas_probable("[CLS] It's not often you find a [MASK] on the street. [SEP]")

BERT input: "[CLS] It's not often you find a [MASK] on the street. [SEP]"
BERT prediction: "car"


'car'

#### **BertForQuestionAnswering**

**Pregunta 3 (1 pto):**  Utilizando el modelo BertForQuestionAnswering pre-entrenado con 'bert-large-uncased-whole-word-masking-finetuned-squad', **extraiga la respuesta** a cada una de las siguientes 4 preguntas y su contexto. **(0.25 puntos cada una)**. Recuerde cambiar el tokenizer para que coincida con el modelo.

In [164]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [206]:
def entregar_respuesta(qst, cntxt):
    #Tu implementacion
    inputs = tokenizer(qst, cntxt, return_tensors='pt')
    outputs = model(**inputs)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Find the slice of the input that contains the answer
    answer_start = start_logits.argmax(dim=1).item()
    answer_end = end_logits.argmax(dim=1).item()

    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    answer_tokens = tokens[answer_start:answer_end+1]

    answer = tokenizer.convert_tokens_to_string(answer_tokens)
    return answer

3.1)

Pregunta: "When was the Battle of Iquique?"

Contexto: "The Battle of Iquique was a naval engagement that occurred between a Chilean corvette under the command of Arturo Prat and a Peruvian ironclad under the command of Miguel Grau Seminario on 21 May 1879, during the naval stage of the War of the Pacific, and resulted in a Peruvian victory."

In [207]:
entregar_respuesta(
    "When was the Battle of Iquique?",
    "The Battle of Iquique was a naval engagement that occurred between a Chilean corvette under the command of Arturo Prat and a Peruvian ironclad under the command of Miguel Grau Seminario on 21 May 1879, during the naval stage of the War of the Pacific, and resulted in a Peruvian victory."
)

'21 may 1879'

3.2)

Pregunta: "Who won the Battle of Iquique?"

Contexto: "The Battle of Iquique was a naval engagement that occurred between a Chilean corvette under the command of Arturo Prat and a Peruvian ironclad under the command of Miguel Grau Seminario on 21 May 1879, during the naval stage of the War of the Pacific, and resulted in a Peruvian victory."

In [208]:
entregar_respuesta(
    "Who won the Battle of Iquique?",
    "The Battle of Iquique was a naval engagement that occurred between a Chilean corvette under the command of Arturo Prat and a Peruvian ironclad under the command of Miguel Grau Seminario on 21 May 1879, during the naval stage of the War of the Pacific, and resulted in a Peruvian victory."
)

'peruvian'

3.3)

Pregunta: "Who introduced peephole connections to LSTM networks?"
Contexto: "What I’ve described so far is a pretty normal LSTM. But not all LSTMs are the same as the above. In fact, it seems like almost every paper involving LSTMs uses a slightly different version. The differences are minor, but it’s worth mentioning some of them. One popular LSTM variant, introduced by Gers & Schmidhuber (2000), is adding “peephole connections.” This means that we let the gate layers look at the cell state."

In [209]:
entregar_respuesta(
    "Who introduced peephole connections to LSTM networks?",
    "What I’ve described so far is a pretty normal LSTM. But not all LSTMs are the same as the above. In fact, it seems like almost every paper involving LSTMs uses a slightly different version. The differences are minor, but it’s worth mentioning some of them. One popular LSTM variant, introduced by Gers & Schmidhuber (2000), is adding “peephole connections.” This means that we let the gate layers look at the cell state."
)

'gers & schmidhuber'

3.4)

Pregunta: "When is the cat most active?"

Contexto: "The cat is similar in anatomy to the other felid species: it has a strong flexible body, quick reflexes, sharp teeth and retractable claws adapted to killing small prey. Its night vision and sense of smell are well developed. Cat communication includes vocalizations like meowing, purring, trilling, hissing, growling and grunting as well as cat-specific body language. It is a solitary hunter but a social species. It can hear sounds too faint or too high in frequency for human ears, such as those made by mice and other small mammals. It is a predator that is most active at dawn and dusk. It secretes and perceives pheromones."

In [210]:
entregar_respuesta(
    "When is the cat most active?",
    "The cat is similar in anatomy to the other felid species: it has a strong flexible body, quick reflexes, sharp teeth and retractable claws adapted to killing small prey. Its night vision and sense of smell are well developed. Cat communication includes vocalizations like meowing, purring, trilling, hissing, growling and grunting as well as cat-specific body language. It is a solitary hunter but a social species. It can hear sounds too faint or too high in frequency for human ears, such as those made by mice and other small mammals. It is a predator that is most active at dawn and dusk. It secretes and perceives pheromones."
)

'dawn and dusk'